# Biodivine AEON

This notebook illustrates how to use Python bindings of the tool AEON to symbolically analyze the attractors of non-trivial Boolean networks.

No special knowledge is necessary to understand the contents of this particular notebook. However, to successfully apply AEON to other problems, some basic understanding of AEON's notion of *partially specified Boolean network* and the underlying symbolic representation using BDDs is recommended (these are explained [here](./bn_tutorial.ipynb) and [here](bdd_tutorial.ipynb)).

We start our exploration by loading a Boolean network from CellCollective:

In [1]:
from biodivine_aeon import *
import biodivine_aeon as ba
from pathlib import Path
import cellcollective

# Load the network from CellCellective.
sbml = cellcollective.load("https://research.cellcollective.org/#module/36604:1/signaling-pathway-for-butanol-production-in-clostridium-beijerinckii-nrrl-b598/10")

# Open the loaded SBML and parse it as a Boolean network.
model = BooleanNetwork.from_file(sbml.localfile)
#model = BooleanNetwork.from_file("cellcollective-36604-1.sbml")
print(f"Loaded model with {model.variable_count()} variables.")

/notebook/tutorials/aeon/cellcollective-36604-1.sbml

Loaded model with 66 variables.


In [2]:
# Print some basic messages (unfortunately, in jupyter notebooks, these may be delayed until the cell finishes computing
# but you can see them in the terminal). ba.LOG_NOTHING is the default, but ba.LOG_VERBOSE is also available.
ba.LOG_LEVEL = ba.LOG_ESSENTIAL

Right now, the model has no explicit paramters, but there are several variables for which no update function is given in the SBML file. AEON will automatically consider these as implicit unknown parameters of the network:

In [3]:
print(f"Number of explicit parameters: {model.explicit_parameter_count()}.")
print(f"Number of implicit parameters: {model.implicit_parameter_count()}.")

Number of explicit parameters: 0.
Number of implicit parameters: 13.


Before we can explore the attractors of this network, first let us observe that the network as downloaded from CellCollective is actually not logically consistent with its regulatory graph. AEON will notify us about this fact when we try to construct the network's asynchronous state-transition graph:

In [4]:
try:
    stg = AsynchronousGraph(model)
except Exception as e:
    print(e)

No update functions satisfy given constraints: 
 - spoIIE not activating in spoIIAB.
 - sporulation has no effect in glucose___PTS.
 - sigA has no effect in spo0A_p.



Fortunately, AEON can tell us exactly which properties are violated. So if we want to fix these issues, we know what variables to focus on. For now, we simply relax the conditions in the regulatory graph such that the existing update functions are valid:

In [5]:
# You can also automatically fix all issues using model.infer_valid_graph()

# Export the Boolean network as `.aeon` model string.
model_aeon = model.to_aeon()

# Relax the problematic regulation constraints:

# Mark regulation "spoIIE -> spoIIAB" as non-monotonous:
model.ensure_regulation({
    'source': 'spoIIE',
    'target': 'spoIIAB',
    'essential': True,
    'sign': None,
})
# Mark regulation "sporulation -| glucose___PTS" as non-essential:
model.ensure_regulation({
    'source': 'sporulation',
    'target': 'glucose___PTS',
    'essential': False,
    'sign': '-',
})
# Mark regulation "sigA -> spo0A_p" as non-essential:
model.ensure_regulation({
    'source': 'sigA',
    'target': 'spo0A_p',
    'essential': False,
    'sign': '+',
})

{'source': VariableId(51),
 'target': VariableId(58),
 'essential': True,
 'sign': '+'}

Now we can actually compute the attractors quite easily. However, note that since AEON has to consider each possible variant of the network, the computation can still take several seconds or minutes (recall that we actually have `2^13 = 8192` possible networks).

In [6]:
stg = AsynchronousGraph(model)
attractors = Attractors.attractors(stg)
attractors

Start interleaved transition guided reduction with 604462909807314600000000[nodes:2] candidates.


[ColoredVertexSet(cardinality=867718363543552, colors=6704, vertices=867718363543552, symbolic_size=5924),
 ColoredVertexSet(cardinality=15959465984, colors=1488, vertices=15959465984, symbolic_size=1393)]

 > Discarded 302231454903657300000000 instances based on the BnVariable(63) extended component.
 > State space reduced to 302231454903657300000000[nodes:5].
 > Discarded 151115727451828650000000 instances based on the BnVariable(51) extended component.
 > State space reduced to 151115727451828650000000[nodes:8].
 > Discarded 75557863725914320000000 instances based on the BnVariable(55) extended component.
 > State space reduced to 75557863725914320000000[nodes:10].
 > Discarded 37778931862957160000000 instances based on the BnVariable(46) extended component.
 > State space reduced to 37778931862957160000000[nodes:13].
 > Discarded 18889465931478580000000 instances based on the BnVariable(45) extended component.
 > State space reduced to 18889465931478580000000[nodes:16].
 > Discarded 9444732965739290000000 instances based on the BnVariable(44) extended component.
 > State space reduced to 9444732965739290000000[nodes:19].
 > Discarded 4722366482869645000000 instances based on the BnVar

As we can see, AEON found two separate attractor sets. Since the sets are very large, only a summary of each set is printed. Each set has millions of vertices and thousands of colors -- AEON uses the term "color" to refer to concrete models arising for different parametrisations.

However, in this case, we can actually verify that these two sets are color-disjoint. This means that each possible network actually still only has one attractor. Consequently, we can just merge the two sets and treat them as a single attractor:

In [7]:
colors_in_both_sets = attractors[0].colors().intersect(attractors[1].colors())
print(f"Number of colors appearing in both sets: {colors_in_both_sets.cardinality()}")

attractor = attractors[0].union(attractors[1])
attractor

Number of colors appearing in both sets: 0


ColoredVertexSet(cardinality=867734323009536, colors=8192, vertices=867734323009536, symbolic_size=6154)

And just as we suspected, there is no intersection between the two sets. This can happen due to how the underlying algorithm works: some stable states are detected first, and more complex attractors are detected afterwards. However, since there are no intersections between these two sets, we can unify them and consider them as one attractor. 

Now, to better understand what is happening in this attractor, we can classify the colors based on their long-term behavioural characteristics (this can also take a few second to compute):

In [8]:
classes = Classification.classify_long_term_behavior(stg, attractor)
classes

{Class(["disorder"]): ColorSet(cardinality=6144, symbolic_size=4),
 Class(["stability"]): ColorSet(cardinality=2048, symbolic_size=4)}

In this case, we can see that 2048 colors correspond to a stable state, and 6144 colors correspond to a disordered attractor (i.e. an attractor that is neither a stable state nor a cycle). In this case, we have no oscillating (cyclic) attractors. 

We can now use these classes to divide the `attractor` set based on its behaviour:

In [9]:
stable_attractor = attractor.intersect_colors(classes[Class("stability")])
disordered_attractor = attractor.intersect_colors(classes[Class("disorder")])

print(stable_attractor)
print(disordered_attractor)

ColoredVertexSet(cardinality=2048, symbolic_size=281)
ColoredVertexSet(cardinality=867734323007488, symbolic_size=5670)


To obtain some useful information from these sets, we can for example look at the possible values of specific network variables within each set. In this model, there is a variable called `sporulation` that is of particular interest, since it signifias that the cell is "hybernating". 

To study whether sporulation is on or off in the possible attractors, we can simply construct the set of all states where sporulation is on, and then compare it to the attractor sets:

In [10]:
sporulation_on = stg.mk_subspace({ "sporulation": True })

on_in_stable_attractor = stable_attractor.intersect(sporulation_on).vertices().cardinality()
off_in_stable_attractor = stable_attractor.minus(sporulation_on).vertices().cardinality()

print("Sporulation is ON in", on_in_stable_attractor, "stable states.")
print("Sporulation is OFF in", off_in_stable_attractor, "stable states.")
print("Sporulation is ON in", round((on_in_stable_attractor / (on_in_stable_attractor + off_in_stable_attractor)) * 100.0, 2), "% of stable attractor states.")

on_in_disorder_attractor = disordered_attractor.intersect(sporulation_on).vertices().cardinality()
off_in_disorder_attractor = disordered_attractor.minus(sporulation_on).vertices().cardinality()

print("Sporulation is ON in", on_in_disorder_attractor, "disorder states.")
print("Sporulation is OFF in", off_in_disorder_attractor, "disorder states.")
print("Sporulation is ON in", round((on_in_disorder_attractor / (on_in_disorder_attractor + off_in_disorder_attractor)) * 100.0, 2), "% of disordered attractor states.")

Sporulation is ON in 2048 stable states.
Sporulation is OFF in 0 stable states.
Sporulation is ON in 100.0 % of stable attractor states.
Sporulation is ON in 335223420223488 disorder states.
Sporulation is OFF in 532510902784000 disorder states.
Sporulation is ON in 38.63 % of disordered attractor states.


Interestingly, we have discovered that in every stable state, `sporulation` is active. We can thus use this fact to pick a witness network which guarantees that eventually, `sporulation` will always be active in a stable attractor. 

This witness network is completely specified (i.e. has no parameters or uninterpreted functions). In other words, all of the explicit and implicit parameters are fixed to guarantee the desired behaviour:

In [11]:
# One color from the color set:
color_model = next(iter(classes[Class("stability")]))
witness = color_model.instantiate(model)

print(f"Number of explicit parameters: {witness.explicit_parameter_count()}.")
print(f"Number of implicit parameters: {witness.implicit_parameter_count()}.")

Number of explicit parameters: 0.
Number of implicit parameters: 0.


Finally, since computing the attractors can take a long time, it may be necessary to save them into a file for further processing. For this, we can export the underlying BDD representation:

In [12]:
Path("stable_attractor.bdd").write_text(stable_attractor.to_bdd().data_string())
Path("disordered_attractor.bdd").write_text(disordered_attractor.to_bdd().data_string())

62415

Or we can use `pickle`:

In [13]:
import pickle
with open("stable_attractors.bdd.pickle", "wb") as f:
    pickle.dump(stable_attractor.to_bdd(), f)
with open("disordered_attractor.bdd.pickle", "wb") as f:
    pickle.dump(disordered_attractor.to_bdd(), f)

With the BDDs saved, we can always reload them from disk. However, remeber that the BDD does not contain any information about the Boolean network from which it was constructed! So you must make sure that you only import BDDs that were created using the same Boolean model. Ideally, always save an `.aeon` or `.sbml` file along with any raw BDD data.

In [14]:
network_context = stg.symbolic_context()
bdd_context = network_context.bdd_variable_set()

stable_reloaded = ColoredVertexSet(network_context, Bdd(bdd_context, Path("stable_attractor.bdd").read_text()))
disordered_reloaded = ColoredVertexSet(network_context, Bdd(bdd_context, Path("disordered_attractor.bdd").read_text()))

print(stable_reloaded)
print(disordered_reloaded)

assert stable_reloaded == stable_attractor
assert disordered_reloaded == disordered_attractor

ColoredVertexSet(cardinality=2048, symbolic_size=281)
ColoredVertexSet(cardinality=867734323007488, symbolic_size=5670)


As we can see, the sets are the same as the ones that we saved into the files.

Finally, let us quickly note that using AEON, you can also compute reachability from/to a specific symbolic set. Furthermore, you can specify a subset within which the reachability should be computed. Using this mechanism, you can often build more complex analysis techniques. For example, you can analyse the strong and weak basin of a particular attractor, or check for reachable attractors from a specific initial state.

In [15]:
# Pick a set containing a single arbitrary vertex from the whole state space:
vertex = stg.mk_unit_colored_vertices().pick_vertex()

# The actual vertex is just a list of Boolean values.
print("Vertex", next(iter(vertex.vertices())).values())

# Compute the set of states backward-reachable from the vertex.
basin = Reachability.reach_bwd(stg, vertex)
print("Basin", basin)

# Compute the strongly-connected-component in which the vertex resides.
scc = Reachability.reach_fwd(stg, vertex).intersect(basin)
print("SCC", scc)

Vertex [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Basin ColoredVertexSet(cardinality=11558771119839541985280, symbolic_size=101)
Start basic symbolic reachability with 8192[nodes:68] candidates.
SCC ColoredVertexSet(cardinality=8192, symbolic_size=68)
Basic reachability done: 11558771119839542000000[nodes:101] candidates. Max intermediate size: 441.
Start basic symbolic reachability with 8192[nodes:68] candidates.
Basic reachability done: 44364707639821350000[nodes:28057] candidates. Max intermediate size: 43163.


For example, for an initial state where all variables are inactive, we can see that its (weak) basin is quite large, but the vertex is in fact a trivial SCC (the whole strongly connected component consists only of one vertex, regardless of color). 


So far, this is the end of our demo. In case of any issues, feel free to contact us on [github](https://github.com/sybila/biodivine-aeon-py)! 

To learn more about what features and functions are available in AEON.py, you can follow the tuturial on [partially specified Boolean networks](./bn_tutorial.ipynb) and [symbolic computation using BDDs](bdd_tutorial.ipynb). Further tutorials, examples and case studies are available in the [AEON.py repository](https://github.com/sybila/biodivine-aeon-py/tree/main/example). API documentation is also available [here](https://biodivine.fi.muni.cz/docs/aeon-py/latest/).